# Venture Funding with Deep Learning

In [1]:
# Import inital libraries and modules
import pandas as pd
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

---

## Prepare the data to be used on a neural network model

In [2]:
# Setup file uploads with Colab
from google.colab import files
uploaded = files.upload()

In [3]:
# Read csv into dataframe - Colab
applicant_data_df = pd.read_csv(Path("./applicants_data.csv"))

# Read csv into dataframe - Non-Colab
# applicant_data_df = pd.read_csv(Path("./Resources/applicants_data.csv"))

# Review the dataframe with .head()
applicant_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Review the data types associated with the columns
applicant_data_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [5]:
# Drop the "EIN" and "NAME" columns from the dataframe
applicant_data_df = applicant_data_df.drop(columns=["EIN", "NAME"], axis=1)

# Review the dataframe with .head()
applicant_data_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Create a list of categorical variables 
categorical_variables = ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"]

# Display the categorical variables list
categorical_variables

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [8]:
# Encode the categorical variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])

In [9]:
# Create a dataframe with the encoded variables
encoded_df = pd.DataFrame(encoded_data, columns=enc.get_feature_names_out(categorical_variables))

# Review the dataframe with .head()
encoded_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Create a list of numerical values
numerical_values = ["STATUS", "ASK_AMT", "IS_SUCCESSFUL"]

# Add the numerical variables from the original dataframe to the OneHotEncoder dataframe
encoded_df = pd.concat([encoded_df, applicant_data_df[numerical_values]], axis=1)

# Review the dataframe with .head()
encoded_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT,IS_SUCCESSFUL
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590,1


In [11]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df["IS_SUCCESSFUL"]

# Display a sample of y
y

0        1
1        1
2        0
3        1
4        1
        ..
34294    0
34295    0
34296    0
34297    1
34298    0
Name: IS_SUCCESSFUL, Length: 34299, dtype: int64

In [12]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns=["IS_SUCCESSFUL"])

# Review the dataframe with .head()
X.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590


In [13]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

In [15]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [16]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [17]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  (number_input_features + number_output_neurons) // 2 

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

58

In [18]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = (hidden_nodes_layer1 + number_output_neurons) // 2 

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

29

In [19]:
# Create the Sequential model instance
nn = Sequential()

In [20]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [21]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [22]:
# Add the output layer
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [23]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Fit the model using 50 epochs and the training data
deep_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 5s 4ms/step - loss: 0.5791 - accuracy: 0.7149
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5526 - accuracy: 0.7285
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5488 - accuracy: 0.7294
Epoch 4/50
804/804 [==============================] - 4s 4ms/step - loss: 0.5474 - accuracy: 0.7322
Epoch 5/50
804/804 [==============================] - 4s 4ms/step - loss: 0.5446 - accuracy: 0.7326
Epoch 6/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5450 - accuracy: 0.7322
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5427 - accuracy: 0.7340
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5427 - accuracy: 0.7337
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5420 - accuracy: 0.7355
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5412 - accuracy: 0.7349

In [26]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5567 - accuracy: 0.7277 - 431ms/epoch - 2ms/step
Loss: 0.5566551089286804, Accuracy: 0.7276967763900757


In [27]:
# Set the model's file path
file_path = Path("./AlphabetSoup.h5")

# Export the model as an HDF5 file
nn.save(file_path)

---

## Optimize the neural network model


### Alternative Model 1

In [38]:
# Alt model 1 - More nodes per layer

# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [39]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

In [40]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (2 / 3) * (number_input_features + number_output_neurons_A1)

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

78.0

In [41]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 = (2 / 3) * hidden_nodes_layer1_A1

# Review the number of hidden nodes in the second layer
hidden_nodes_layer2_A1

52.0

In [42]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [43]:
# Add the first hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, input_dim=number_input_features, activation="relu"))

# Add the Second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))

# Add the output layer
nn_A1.add(Dense(units=number_output_neurons_A1, activation="sigmoid"))

# Display the Sequential model summary
nn_A1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 78)                9126      
                                                                 
 dense_9 (Dense)             (None, 52)                4108      
                                                                 
 dense_10 (Dense)            (None, 1)                 53        
                                                                 
Total params: 13,287
Trainable params: 13,287
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Compile the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [46]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5304 - accuracy: 0.7417
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5310 - accuracy: 0.7416
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5307 - accuracy: 0.7422
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5304 - accuracy: 0.7416
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5300 - accuracy: 0.7420
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5301 - accuracy: 0.7420
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5302 - accuracy: 0.7423
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5302 - accuracy: 0.7417
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5300 - accuracy: 0.7424
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5301 - accuracy: 0.7414

#### Alternative Model 2

In [47]:

# Alt model 2 - More hidden layers

# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [48]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [49]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = (number_input_features + number_output_neurons_A2) // 2 

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

58

In [50]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 = hidden_nodes_layer1_A2 // 2 

# Review the number of hidden nodes in the second layer
hidden_nodes_layer2_A2

29

In [51]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A2 = hidden_nodes_layer2_A2 // 2 

# Review the number of hidden nodes in the third layer
hidden_nodes_layer3_A2

14

In [52]:
# Define the number of hidden nodes for the fourth hidden layer
hidden_nodes_layer4_A2 = hidden_nodes_layer3_A2 // 2 

# Review the number of hidden nodes in the fourth layer
hidden_nodes_layer4_A2

7

In [53]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [54]:
# Add the first hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, input_dim=number_input_features, activation="relu"))


# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A2, activation="relu"))


# Add the third hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer3_A2, activation="relu"))


# Add the fourth hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer4_A2, activation="relu"))


# Add the output layer
nn_A2.add(Dense(units=number_output_neurons_A2, activation="sigmoid"))


# Display the Sequential model summary
nn_A2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 58)                6786      
                                                                 
 dense_12 (Dense)            (None, 29)                1711      
                                                                 
 dense_13 (Dense)            (None, 14)                420       
                                                                 
 dense_14 (Dense)            (None, 7)                 105       
                                                                 
 dense_15 (Dense)            (None, 1)                 8         
                                                                 
Total params: 9,030
Trainable params: 9,030
Non-trainable params: 0
_________________________________________________________________


In [55]:
# Compile the Sequential model
nn_A2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [56]:
# Fit the model using 50 epochs and the training data
fit_model_A2 = nn_A2.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5795 - accuracy: 0.7148
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7295
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7317
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7338
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7331
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accuracy: 0.7345
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accuracy: 0.7341
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5429 - accuracy: 0.7351
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5419 - accuracy: 0.7349
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5415 - accuracy: 0.7360

#### Display and Compare Results

In [66]:
# Print out evaluation metrics for original model
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss:.6f}, Accuracy: {model_accuracy:.6f}")

Original Model Results
268/268 - 1s - loss: 0.5567 - accuracy: 0.7277 - 629ms/epoch - 2ms/step
Loss: 0.556655, Accuracy: 0.727697


In [67]:
# Print out evaluation metrics for alternative model 1
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss:.6f}, Accuracy: {model_accuracy:.6f}")

Alternative Model 1 Results
268/268 - 1s - loss: 0.5689 - accuracy: 0.7297 - 555ms/epoch - 2ms/step
Loss: 0.568937, Accuracy: 0.729679


In [68]:
# Print out evaluation metrics for alternative model 2
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss:.6f}, Accuracy: {model_accuracy:.6f}")

Alternative Model 2 Results
268/268 - 1s - loss: 0.5567 - accuracy: 0.7308 - 502ms/epoch - 2ms/step
Loss: 0.556719, Accuracy: 0.730845


In [60]:
# Set the file path for the first alternative model
file_path = Path("./AlphabetSoupAlt1.h5")

# Export the model as an HDF5 file
nn_A1.save(file_path)

In [61]:
# Set the file path for the second alternative model
file_path = Path("./AlphabetSoupAlt2.h5")

# Export the model as an HDF5 file
nn_A2.save(file_path)